In [339]:
import pandas as pd
from itertools import combinations
import math
import os
import csv
import time

In [340]:
cargo_capacity_psv = 100
psv_speed = 10 #7 for strøm, 10 på resten
max_platforms_in_one_voyage = 6

## Reads input data

In [341]:
demand = pd.read_csv('clustering/output_platforms_demand.csv', header=0, delimiter=';')
distances = pd.read_csv('clustering/output_distance_matrix_kmeans.csv', header=0, delimiter=';', index_col='from/to')

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['MON'] + demand['platform'].tolist() + ['MON']  # Add 'Mon' as start and end platforms

## Algorithm for generating the shortest routes

In [342]:
shortest_routes_dict = {}

def generate_routes(demand, distances):
    cargo_capacity = cargo_capacity_psv
    max_platforms = max_platforms_in_one_voyage + 2

    def dp(platform, cargo_remaining, route, visited):
        if cargo_remaining < 0:
            return
        if len(route) > max_platforms:
            return
        if platform == 'MON' and len(route) > 2:
            total_demand = sum(platforms_demand[p] for p in route[1:-1])
            if total_demand <= cargo_capacity:
                key = tuple(sorted(set(route)))
                total_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
                if key not in shortest_routes_dict or total_distance < shortest_routes_dict[key][1]:
                    duration_sailing = round((total_distance / psv_speed), 2)
                    duration_lossing = round(((total_demand * 1.389) / psv_speed), 2)
                    duration_sailing = round(duration_sailing, 2)
                    duration_lossing = round(duration_lossing, 2)
                    shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            return

        # Check if the current route is dominated
        current_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
        current_demand = sum(platforms_demand[p] for p in route[1:-1])
        
        # Check for dominance in existing routes
        for key, (existing_route, existing_distance, existing_demand, _, _) in shortest_routes_dict.items():
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return
        
        # Check for dominance in subsequent routes starting with the same platforms
        for existing_route in dominated_routes:
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return

        for next_platform in platforms_demand.keys():
            if next_platform != platform and next_platform not in visited:
                try:
                    distance_to_next = distances.loc[platform, next_platform]
                    new_cargo_remaining = cargo_remaining - platforms_demand[next_platform]
                    dp(next_platform, new_cargo_remaining, route + [next_platform], visited.union({next_platform}))
                except KeyError:
                    print("KeyError occurred for platform:", next_platform)
                    continue

    for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):
        for route_combination in combinations(platforms_demand.keys(), r - 2):
            route = ['MON'] + list(route_combination) + ['MON']
            dp('MON', cargo_capacity, route, {'MON'})

            # Add the current route to the dominated routes set
            dominated_routes.add(tuple(route))

    return shortest_routes_dict

# Keep track of dominated routes to skip future routes starting with them
dominated_routes = set()
shortest_routes_dict = generate_routes(demand, distances)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {round(distance,2)}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print(len(shortest_routes_dict))

Shortest Route: ['MON', 'APT', 'MON'], Total Distance: 148.32, Total Demand: 18.0, Duration sailing: 14.83, Duration lossing: 2.5
Shortest Route: ['MON', 'ASL', 'MON'], Total Distance: 179.48, Total Demand: 25.0, Duration sailing: 17.95, Duration lossing: 3.47
Shortest Route: ['MON', 'DAB', 'MON'], Total Distance: 110.98, Total Demand: 40.0, Duration sailing: 11.1, Duration lossing: 5.56
Shortest Route: ['MON', 'DSA', 'MON'], Total Distance: 194.44, Total Demand: 29.0, Duration sailing: 19.44, Duration lossing: 4.03
Shortest Route: ['MON', 'DSS', 'MON'], Total Distance: 189.78, Total Demand: 29.0, Duration sailing: 18.98, Duration lossing: 4.03
Shortest Route: ['MON', 'GFAGFBGFC', 'MON'], Total Distance: 176.96, Total Demand: 67.25, Duration sailing: 17.7, Duration lossing: 9.34
Shortest Route: ['MON', 'KVB', 'MON'], Total Distance: 151.06, Total Demand: 14.0, Duration sailing: 15.11, Duration lossing: 1.94
Shortest Route: ['MON', 'MID', 'MON'], Total Distance: 181.28, Total Demand: 14

## Checks the capacity utilization (demand delivered). Removes all routes with capacity utilization below 70%

In [343]:
count_below_10 = 0
count_below_20 = 0
count_below_30 = 0
count_below_40 = 0
count_below_50 = 0
count_below_60 = 0
count_below_70 = 0
count_below_80 = 0
count_below_90 = 0

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    if demand < 10:
        count_below_10 += 1
    if demand < 20:
        count_below_20 += 1
    if demand < 30:
        count_below_30 += 1
    if demand < 40:
        count_below_40 += 1
    if demand < 50:
        count_below_50 += 1
    if demand < 60:
        count_below_60 += 1
    if demand < 70:
        count_below_70 += 1
    if demand < 80:
        count_below_80 += 1
    if demand < 90:
        count_below_90 += 1

print("Number of shortest routes with total demand below 10:", count_below_10)
print("Number of shortest routes with total demand below 20:", count_below_20)
print("Number of shortest routes with total demand below 30:", count_below_30)
print("Number of shortest routes with total demand below 40:", count_below_40)
print("Number of shortest routes with total demand below 50:", count_below_50)
print("Number of shortest routes with total demand below 60:", count_below_60)
print("Number of shortest routes with total demand below 70:", count_below_70)
print("Number of shortest routes with total demand below 80:", count_below_80)
print("Number of shortest routes with total demand below 90:", count_below_90)

Number of shortest routes with total demand below 10: 0
Number of shortest routes with total demand below 20: 6
Number of shortest routes with total demand below 30: 20
Number of shortest routes with total demand below 40: 40
Number of shortest routes with total demand below 50: 97
Number of shortest routes with total demand below 60: 215
Number of shortest routes with total demand below 70: 403
Number of shortest routes with total demand below 80: 769
Number of shortest routes with total demand below 90: 1363


In [344]:
shortest_routes_dict_cap = {}

for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict.items():
    if demand > 70:
        shortest_routes_dict_cap[key] = (route, distance, demand, duration_sailing, duration_lossing)

In [345]:
print(len(shortest_routes_dict_cap))

1818


## Removes routes which results in very long idle time

In [346]:
count_between_16_and_24 = 0
count_between_40_and_48 = 0

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict_cap.values():
    total_duration = duration_sailing + duration_lossing
    if 16 < total_duration < 24:
        count_between_16_and_24 += 1
    if 40 < total_duration < 48:
        count_between_40_and_48 += 1

print("Count of routes with duration between 16 and 24:", count_between_16_and_24)
print("Count of routes with duration between 40 and 48:", count_between_40_and_48)

Count of routes with duration between 16 and 24: 0
Count of routes with duration between 40 and 48: 344


In [347]:
shortest_routes_dict_cap_idle = {}
routes_to_remove = []

for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict_cap.items():
    total_duration = duration_sailing + duration_lossing
    if 16 < total_duration < 24 or 40 < total_duration < 48:
        routes_to_remove.append(key)
    else:
        shortest_routes_dict_cap_idle[key] = (route, distance, demand, duration_sailing, duration_lossing)

for key in routes_to_remove:
    del shortest_routes_dict_cap[key]

In [348]:
print(len(shortest_routes_dict_cap))

1474


## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 2294 routes

In [349]:
routes_only = [route for route, _, _, _, _ in shortest_routes_dict_cap.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict_cap.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict_cap.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict_cap.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict_cap.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv('generated_datafiles/distances_strøm.csv', index=False)
df_demand.to_csv('generated_datafiles/demand_strøm.csv', index=False)
df_duration_sailing.to_csv('generated_datafiles/duration_sailing_strøm.csv', index=False)
df_duration_lossing.to_csv('generated_datafiles/duration_lossing_strøm.csv', index=False)

route_file = "generated_datafiles/routes_strøm.csv"

def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)